
# おおよそのトレーニング

:label: `sec_approx_train`

 :numref: `sec_word2vec`での議論を思い出してください。スキップグラム モデルの主なアイデアは、ソフトマックス演算を使用して、 :eqref: `eq_skip-gram-softmax`内の指定された中心単語 $w_c$ に基づいて文脈単語 $w_o$ を生成する条件付き確率を計算することです。その対応する対数損失は次のとおりです。 :eqref: `eq_skip-gram-log`の反対によって与えられます。

ソフトマックス演算の性質上、コンテキストワードは辞書 $\mathcal{V}$ 内の任意の単語になる可能性があるため、:eqref: `eq_skip-gram-log`の逆には、全体のサイズと同じ数の項目の合計が含まれます。語彙。したがって、:eqref: `eq_skip-gram-grad`のスキップグラム モデルの勾配計算と、:eqref: `eq_cbow-gradient`の連続バッグオブワード モデルの勾配計算の両方に合計が含まれます。残念ながら、このような勾配の計算コストは​​、合計すると大きな辞書 (多くの場合、数十万語または数百万語の単語を含む) を超えるほど膨大になります。

前述の計算の複雑さを軽減するために、このセクションでは、*ネガティブ サンプリング*と*階層的ソフトマックス*という 2 つの近似トレーニング方法を紹介します。スキップグラム モデルと連続バッグ オブ ワード モデルは類似しているため、これら 2 つの近似トレーニング方法を説明する例としてスキップグラム モデルのみを取り上げます。

## ネガティブサンプリング

:label: `subsec_negative-sampling`

ネガティブサンプリングでは、元の目的関数が変更されます。中心単語 $w_c$ の文脈ウィンドウが与えられると、任意の (文脈) 単語 $w_o$ がこの文脈ウィンドウから来るという事実は、次のようにモデル化された確率を持つイベントと見なされます。

 $$P(D=1\mid w_c, w_o) = \sigma(\mathbf{u}_o^\top \mathbf{v}_c),$$

ここで、$\sigma$ はシグモイド活性化関数の定義を使用します。

 $$\sigma(x) = \frac{1}{1+\exp(-x)}.$$ :eqlabel: `eq_sigma-f`

単語の埋め込みをトレーニングするために、テキスト シーケンス内のすべてのそのようなイベントの同時確率を最大化することから始めましょう。具体的には、長さ $T$ のテキスト シーケンスが与えられた場合、タイム ステップ $t$ の単語を $w^{(t)}$ で表し、コンテキスト ウィンドウ サイズを $m$ として、同時確率を最大化することを検討します。

 $$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(D=1\mid w^{(t)}, w^{( t+j)}).$$ :eqlabel: `eq-negative-sample-pos`

ただし、:eqref: `eq-negative-sample-pos`正の例を含むイベントのみを考慮します。その結果、:eqref: `eq-negative-sample-pos`の結合確率は、すべての単語ベクトルが無限大に等しい場合にのみ 1 に最大化されます。もちろん、そのような結果には意味がありません。目的関数をより意味のあるものにするために、*負のサンプリングでは*、事前定義された分布からサンプリングされた負の例が追加されます。

 $S$ は、文脈単語 $w_o$ が中心単語 $w_c$ の文脈ウィンドウから来るというイベントを示します。 $w_o$ が関係するこのイベントでは、事前定義された分布 $P(w)$ から、このコンテキスト ウィンドウからではない $K$*のノイズ ワード*がサンプリングされます。 $N_k$ は、ノイズ ワード $w_k$ ($k=1, \ldots, K$) が $w_c$ のコンテキスト ウィンドウから来ていないイベントを示します。正の例と負の例 $S、N_1、\ldots、N_K$ の両方を含むこれらのイベントは相互に独立していると仮定します。負のサンプリングでは、:eqref: `eq-negative-sample-pos`の同時確率 (正の例のみを含む) が次のように書き換えられます。

 $$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t) })、$$

ここで、条件付き確率はイベント $S、N_1、\ldots、N_K$ を通じて近似されます。

 $$ P(w^{(t+j)} \mid w^{(t)}) =P(D=1\mid w^{(t)}, w^{(t+j)})\ prod_{k=1,\ w_k \sim P(w)}^KP(D=0\mid w^{(t)}, w_k).$$ :eqlabel: `eq-negative-sample-conditional-prob`

テキストシーケンスのタイムステップ $t$ における単語 $w^{(t)}$ とノイズワード $w_k$ のインデックスをそれぞれ $i_t$ と $h_k$ で表します。 :eqref: `eq-negative-sample-conditional-prob`の条件付き確率に関する対数損失は次のとおりです。

 $$ \begin{aligned} -\log P(w^{(t+j)} \mid w^{(t)}) =&amp; -\log P(D=1\mid w^{(t)} , w^{(t+j)}) - \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\ =&amp;- \log, \sigma\left(\mathbf{u} *{i* {t+j}}^\top \mathbf{v} *{i_t}\right) - \sum* {k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\mathbf{u} *{h_k}^\top \mathbf{v}* {i_t}\right)\right)\ =&amp;- \log , \sigma\left(\mathbf{u} *{i* {t+j}}^\top \mathbf{v} *{i_t}\right) - \sum* {k=1,\ w_k \sim P(w)} ^K \log\sigma\left(-\mathbf{u} *{h_k}^\top \mathbf{v}* {i_t}\right)。 \end{整列} $$

これで、各トレーニング ステップでの勾配の計算コストは​​辞書のサイズとは関係がなく、$K$ に線形的に依存することがわかります。ハイパーパラメーター $K$ をより小さい値に設定すると、負のサンプリングを使用した各トレーニング ステップでの勾配の計算コストが小さくなります。

## 階層型ソフトマックス

代替の近似トレーニング方法として、*階層ソフトマックスは*:numref: `fig_hi_softmax`で示されるデータ構造であるバイナリ ツリーを使用します。ここで、ツリーの各葉ノードは辞書 $\mathcal{V}$ 内の単語を表します。 

![](../img/hi-softmax.svg) :ラベル: `fig_hi_softmax`

二分木において単語$w$を表す根ノードから葉ノードまでの経路上のノード数(両端を含む)を$L(w)$で表す。 $n(w,j)$ をこのパス上の $j^\mathrm{th}$ ノードとし、そのコンテキストワードベクトルを $\mathbf{u}_{n(w, j)}$ とします。たとえば、 :numref: `fig_hi_softmax`の $L(w_3) = 4$ です。階層的ソフトマックスは、:eqref: `eq_skip-gram-softmax`の条件付き確率を次のように近似します。

 $$P(w_o \mid w_c) = \prod_{j=1}^{L(w_o)-1} \sigma\left( [![ n(w_o, j+1) = \text{leftChild}(n (w_o, j)) ]!] \cdot \mathbf{u}_{n(w_o, j)}^\top \mathbf{v}_c\right),$$

ここで、関数 $\sigma$ は :eqref: `eq_sigma-f`で定義されており、 $\text{leftChild}(n)$ はノード $n$ の左の子ノードです。 $x$ が true の場合、$[![x ]!] = 1$;それ以外の場合は $[![x]!] = -1$。

説明のために、 :numref: `fig_hi_softmax`の単語 $w_c$ から単語 $w_3$ を生成する条件付き確率を計算してみましょう。これには、ルートから $w_3$ までのパス ( :numref: `fig_hi_softmax`の太字のパス) 上の $w_c$ のワード ベクトル $\mathbf{v}_c$ と非リーフ ノード ベクトルの間の内積が必要です。左、右、そして左と横断しました:

 $$P(w_3 \mid w_c) = \sigma(\mathbf{u} *{n(w_3, 1)}^\top \mathbf{v}_c) \cdot \sigma(-\mathbf{u} {* n( w_3, 2)}^\top \mathbf{v} *c) \cdot \sigma(\mathbf{u}* {n(w_3, 3)}^\top \mathbf{v}_c).$$

 $\sigma(x)+\sigma(-x) = 1$ なので、任意の単語 $w_c$ に基づいて辞書 $\mathcal{V}$ 内のすべての単語を生成する条件付き確率の合計は 1 になることがわかります。

 $$\sum_{w \in \mathcal{V}} P(w \mid w_c) = 1.$$ :eqlabel: `eq_hi-softmax-sum-one`

幸いなことに、$L(w_o)-1$ はバイナリ ツリー構造により $\mathcal{O}(\text{log}_2|\mathcal{V}|)$ 程度であるため、辞書サイズが $ の場合、 \mathcal{V}$ は膨大ですが、階層的ソフトマックスを使用した各トレーニング ステップの計算コストは​​、近似トレーニングを行わない場合と比較して大幅に削減されます。

## まとめ
- ネガティブ サンプリングでは、正の例と負の例の両方を含む相互に独立したイベントを考慮して損失関数を構築します。トレーニングの計算コストは​​、各ステップのノイズ ワードの数に線形的に依存します。
- 階層ソフトマックスは、バイナリ ツリーのルート ノードからリーフ ノードまでのパスを使用して損失関数を構築します。トレーニングの計算コストは​​、各ステップの辞書サイズの対数に依存します。

## 演習
1. ネガティブサンプリングでノイズワードをサンプリングするにはどうすればよいでしょうか?
1.  :eqref: `eq_hi-softmax-sum-one`が成立することを確認します。
1. ネガティブサンプリングと階層ソフトマックスをそれぞれ使用して連続バッグオブワードモデルをトレーニングするにはどうすればよいですか?

[ディスカッション](https://discuss.d2l.ai/t/382)
